# Homework 2, Step 2. CNN for CIFAR-100

In the step 2, you need to try CNN on the CIFAR-100 classification.

## 1. Prepare the dataset and the model
### Import Library

In [1]:
import os
import sys

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import numpy as np
import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter

import time

import tqdm

### Hyperparameters
**For step 3, 4, 5,** you need to change the value of momentum, weight decay, data augmentation and batch normalization, to see the difference.

`mmt`: momentum for the optimizer. Use `0` if you do not want to use the momentum.

`wd`: weight decay for the optimizer. Use `0` if you do not want to use the weight decay.

`data_augmentation`: whether to use the data augmentation for the training.

`use_BN`: wheter to use the batch normalization for the training. 

In [2]:
lr = 0.1  # learning rate
opt = 'sgd'
batchsize = 256  # training batchsize

mmt = 0.9  # momentum for optimizer
wd = 5e-4  # weight_decay for optimizer
data_augmentation = True
use_BN = False

### Load Dataset
In pytorch, you can use the following API to load the dataset.

The RGB mean and std are pre-calculated values for normalizing the data. **Do not modify them**.

In [3]:
rgb_mean = np.array([0.4914, 0.4822, 0.4465])
rgb_std = np.array([0.2023, 0.1994, 0.2010])
if data_augmentation:
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(rgb_mean, rgb_std),
    ])
else:
    transform_train = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(rgb_mean, rgb_std),
    ])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(rgb_mean, rgb_std),
])

trainset = torchvision.datasets.CIFAR100(
    root='../data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=batchsize, shuffle=True, num_workers=1)

testset = torchvision.datasets.CIFAR100(
    root='../data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=500, shuffle=False, num_workers=1)

Files already downloaded and verified
Files already downloaded and verified


### Define the CNN Model

In [4]:
class Identity(nn.Module):
    def __init__(self) -> None:
        super().__init__()

    def forward(self, x):
        return x


def _weights_init(m):
    if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight.data)


def compute_feature_map_size(ks, pks):
    s = 32
    for k, pk in zip(ks, pks):
        s = s-k+1
        s = int(s/pk)
    return s


class LeNet(nn.Module):
    def __init__(self, num_classes, hidden_chns, ks, pks):
        super(LeNet, self).__init__()
        in_chns, out_chns = [3, *hidden_chns[:-1]], hidden_chns
        layers = []
        for ic, oc, k, pk in zip(in_chns, out_chns, ks, pks):
            layers.append(nn.Conv2d(ic, oc, kernel_size=k))
            if use_BN:
                layers.append(nn.BatchNorm2d(oc))
            layers.append(nn.MaxPool2d(kernel_size=pk, stride=2) if pk>1 else Identity()) 
            layers.append(nn.ReLU(inplace=True))
        self.layers = nn.Sequential(*layers)

        s = compute_feature_map_size(ks, pks)
        print(f"feature size: {s}")

#         self.fc = nn.Linear(hidden_chns[-1] * s * s, num_classes)
        self.fc = nn.Sequential(
            nn.Linear(hidden_chns[-1] * s * s, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        x = self.layers(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)


def lenet(hidden_chns, ks, pks):
    return LeNet(num_classes=100, hidden_chns=hidden_chns, ks=ks, pks=pks)

## 2. Define the model and run

### Training settings
You may modify the `num_epochs` for the fast training or the better performance.

In [5]:
num_epochs = 150  # training epochs
best_acc = 0.0  # best accuracy

### Define the model, optimizer, loss function, learning rate scheduler
For step 2, 3, 4, 5, you need to change the network structure of the CNN.

`hidden_chns`: a list of the hidden channels of the conv layer.

`ks`: a list of the kernel sizes of the conv layer.

`pks`: a list of the pooling kernel sizes of the pooling layer. Use `1` if you do not want to use a pooling layer (and it will be an identity function).

In [6]:
net = lenet(
    hidden_chns=[128, 128, 256, 256, 512],
    ks=[3, 3, 3, 3, 3],
    pks=[1, 2, 1, 2, 2]
)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
net = net.to(device)  # put the model on the specified device(e.g. gpu/cpu)

# loss function
criterion = nn.CrossEntropyLoss()

# optimizer
optimizer = optim.SGD(net.parameters(), lr=lr, momentum=mmt, weight_decay=wd)  # momentum

# learning rate scheduler
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

feature size: 1
cuda


### Check the total number of parameters

In [7]:
def count_params(net):
    # you can use this function to count amount of your model parameters
    import numpy as np
    total_params = 0

    for x in filter(lambda p: p.requires_grad, net.parameters()):
        total_params += np.prod(x.data.cpu().numpy().shape)
    print("Total number of params", total_params)
    print("Total layers", len(list(filter(lambda p: p.requires_grad and len(p.data.size())>1, net.parameters()))))


count_params(net)

Total number of params 21508836
Total layers 8


### Training logs
The training logs are saved in the `exp` folder. You can use tensorboard to see the logs.

In [8]:
if not os.path.exists("exp"):
    os.mkdir("exp")
last_train = max([eval(s.split("-")[-1]) for s in os.listdir("exp")] + [0])
current_train = last_train + 1
save_dir = "exp/cifar100-{}".format(current_train)
os.makedirs(save_dir)
writer = SummaryWriter(save_dir)

best_acc = 0

### Training and testing

In [9]:
# Training
def train(epoch):
    net.train()
    train_loss = 0
    correct = 0
    total = 0

    with tqdm.tqdm(enumerate(trainloader), total=len(trainloader)) as t:
        t.set_description(f"Epoch {epoch} train")
        for batch_idx, (inputs, targets) in t:
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            t.set_postfix(
                {
                    "loss": f"{train_loss/(batch_idx+1):.3f}",
                    "acc": f"{100.*correct/total:.3f}%, {correct}/{total}",
                }
            )

    writer.add_scalars(
        "loss", {"train": train_loss/len(trainloader)},
        global_step=epoch * len(trainloader)
    )
    writer.add_scalars(
        "accuracy", {"train": correct/total},
        global_step=epoch
    )


# validation
def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        with tqdm.tqdm(enumerate(testloader), total=len(testloader)) as t:
            t.set_description(f"Epoch {epoch}  test")
            for batch_idx, (inputs, targets) in t:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = net(inputs)
                loss = criterion(outputs, targets)

                test_loss += loss.item()
                _, predicted = outputs.max(1)
                total += targets.size(0)
                correct += predicted.eq(targets).sum().item()

                t.set_postfix(
                    {
                        "loss": f"{test_loss / (batch_idx + 1):.3f}",
                        "acc": f"{correct*100./total:.3f}%, {correct}/{total}",
                    }
                )

    writer.add_scalars(
        "loss", {"test": test_loss/len(testloader)},
        global_step=epoch * len(trainloader)
    )
    writer.add_scalars(
        "accuracy", {"test": correct/total},
        global_step=epoch
    )

    # Save checkpoint.
    acc = 100.0 * correct / total
    if acc > best_acc:
        print("Saving..")
        state = {
            "net": net.state_dict(),
            "acc": acc,
            "epoch": epoch,
        }
        torch.save(state, os.path.join(save_dir, "ckpt.pth"))
        best_acc = acc

    return acc


for epoch in range(0, num_epochs):
    tic = time.time()
    train(epoch)
    test_acc = test(epoch)
    t = time.time() - tic
    print(
        f"Epoch {epoch} | total time: {t:.0f}s, test acc: {test_acc:.3f}%, best acc: {best_acc:.3f}%"
    )
    scheduler.step()

Epoch 0 train:   0%|          | 0/196 [00:00<?, ?it/s]/opt/conda/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
Epoch 0  test: 100%|██████████| 20/20 [00:02<00:00,  6.81it/s, loss=4.208, acc=5.300%, 530/10000]


Saving..
Epoch 0 | total time: 24s, test acc: 5.300%, best acc: 5.300%


Epoch 1  test: 100%|██████████| 20/20 [00:03<00:00,  6.13it/s, loss=3.886, acc=9.130%, 913/10000]


Saving..
Epoch 1 | total time: 24s, test acc: 9.130%, best acc: 9.130%


Epoch 2  test: 100%|██████████| 20/20 [00:03<00:00,  5.98it/s, loss=3.530, acc=14.690%, 1469/10000]


...

Epoch 147 | total time: 24s, test acc: 62.670%, best acc: 63.190%


Epoch 148  test: 100%|██████████| 20/20 [00:03<00:00,  5.78it/s, loss=1.691, acc=63.410%, 6341/10000]


Saving..
Epoch 148 | total time: 25s, test acc: 63.410%, best acc: 63.410%


Epoch 149  test: 100%|██████████| 20/20 [00:03<00:00,  6.04it/s, loss=1.671, acc=63.970%, 6397/10000]


Saving..
Epoch 149 | total time: 24s, test acc: 63.970%, best acc: 63.970%
